In [1]:
import ee
import geemap

In [17]:
js_snippet = """

function projectShadows(cloudMask,image,cloudHeights,dilatePixels){
  var meanAzimuth = image.get('SUN_AZIMUTH');
  var meanZenith1 = image.get('SUN_ELEVATION');
  var rightangle = ee.Number(90);
  var meanZenith = rightangle.subtract(meanZenith1);
  ///////////////////////////////////////////////////////
  
  //Find dark pixels
  var darkPixels = image.select(['nir','swir1','swir2']).reduce(ee.Reducer.sum()).lt(irSumThresh);//.gte(1);
  
  //Get scale of image
  var nominalScale = cloudMask.projection().nominalScale();

  //Find where cloud shadows should be based on solar geometry
  //Convert to solar geometry to radians
  var azR =ee.Number(meanAzimuth).multiply(Math.PI).divide(180.0).add(ee.Number(0.5).multiply(Math.PI ));
  var zenR  =ee.Number(0.5).multiply(Math.PI ).subtract(ee.Number(meanZenith).multiply(Math.PI).divide(180.0));
  
  //Find the shadows
  var shadows = cloudHeights.map(function(cloudHeight){
    cloudHeight = ee.Number(cloudHeight);
    
    var shadowCastedDistance = zenR.tan().multiply(cloudHeight);//Distance shadow is cast
    var x = azR.cos().multiply(shadowCastedDistance).divide(nominalScale).round();//X distance of shadow
    var y = azR.sin().multiply(shadowCastedDistance).divide(nominalScale).round();//Y distance of shadow
    return cloudMask.changeProj(cloudMask.projection(), cloudMask.projection().translate(x, y));
  });

  var shadowMask = ee.ImageCollection.fromImages(shadows).max();

  //Create shadow mask
  shadowMask = shadowMask.and(cloudMask.not());
  shadowMask = shadowMask.focal_min(contractPixels).focal_max(dilatePixels);

  shadowMask = shadowMask.and(darkPixels);
  
  var cloudShadowMask = shadowMask; //.or(cloudMask);
  
  image = image.addBands(shadowMask.rename(['cloudShadow'])); //.updateMask(cloudShadowMask.not()).addBands(shadowMask.rename(['cloudShadow']));
  
  return image;
}

"""

In [18]:
geemap.js_snippet_to_py(
    js_snippet
)

In [ ]:
import ee
import geemap

import math

Map = geemap.Map()

def projectShadows(cloudMask,image,cloudHeights,dilatePixels):
  meanAzimuth = image.get('SUN_AZIMUTH')
  meanZenith1 = image.get('SUN_ELEVATION')
  rightangle = ee.Number(90)
  meanZenith = rightangle.subtract(meanZenith1)
  ###########################/

  #Find dark pixels
  darkPixels = image.select(['nir','swir1','swir2']).reduce(ee.Reducer.sum()).lt(irSumThresh);#.gte(1)

  #Get scale of image
  nominalScale = cloudMask.projection().nominalScale()

  #Find where cloud shadows should be based on solar geometry
  #Convert to solar geometry to radians
  azR =ee.Number(meanAzimuth).multiply(math.pi).divide(180.0).add(ee.Number(0.5).multiply(math.pi ))
  zenR  =ee.Number(0.5).multiply(math.pi ).subtract(ee.Number(meanZenith).multiply(math.pi).divide(180.0))

  #Find the shadows

def func_hgd(cloudHeight):
    cloudHeight = ee.Number(cloudHeight)

    shadowCastedDistance = zenR.tan().multiply(cloudHeight);#Distance shadow is cast
    x = azR.cos().multiply(shadowCastedDistance).divide(nominalScale).round();#X distance of shadow
    y = azR.sin().multiply(shadowCastedDistance).divide(nominalScale).round();#Y distance of shadow
    return cloudMask.changeProj(cloudMask.projection(), cloudMask.projection().translate(x, y))

  shadows = cloudHeights.map(func_hgd)

  shadowMask = ee.ImageCollection.fromImages(shadows).max()

  #Create shadow mask
  shadowMask = shadowMask.And(cloudMask.Not())
  shadowMask = shadowMask.focal_min(contractPixels).focal_max(dilatePixels)

  shadowMask = shadowMask.And(darkPixels)

  cloudShadowMask = shadowMask; #.Or(cloudMask)

  image = image.addBands(shadowMask.rename(['cloudShadow'])); #.updateMask(cloudShadowMask.Not()).addBands(shadowMask.rename(['cloudShadow']))

  return image

Map
